In [1]:
import os

# 设置工作目录为项目的主目录
os.chdir("../")  # 使用相对路径将工作目录切换到 project 文件夹
print("Current working directory:", os.getcwd())

Current working directory: /home/yixiong_ren/work/FBM


In [2]:
import torch
import torch.nn as nn
from utils import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
set_seed(42)

Using device: cuda


# 测试单层FBM准确性

In [3]:
#train_loader, test_loader = load_cifar10()
#selected_classes = [0, 1]
class_counts = [100]*10
datatype = 'MNIST'

train_loader = get_dataloader(datatype, batch_size=64, train=True, class_counts=class_counts)
test_loader = get_dataloader(datatype, batch_size=64, train=False)

data_iter = iter(train_loader)
images, labels = next(data_iter)
batch, channel, large, _ = images.shape

In [4]:
import torch.optim as optim
import torch.nn.functional as F
from train.train import train_FBM

# 定义超参数
input_size = channel * large**2 # MNIST图像大小是28x28
hidden_dim =1000
num_classes = 10      # MNIST有10个类别
learning_rate = 0.01
lam = 0.01
d_f = 0.43
alpha = 7.0
num_epochs = 10
batch_size = 64

model = train_FBM(input_size = input_size, 
                output_size = hidden_dim, 
                lam = lam, 
                d_f = d_f,
                alpha = alpha, 
                learning_rate = learning_rate, 
                train_loader = train_loader, 
                num_epochs = num_epochs, 
                device = device)

Epoch [1/10], Loss: 0.7034
Epoch [2/10], Loss: 0.5333
Epoch [3/10], Loss: 0.4682
Epoch [4/10], Loss: 0.5532
Epoch [5/10], Loss: 0.4657
Epoch [6/10], Loss: 0.4832
Epoch [7/10], Loss: 0.4295
Epoch [8/10], Loss: 0.4564
Epoch [9/10], Loss: 0.4276
Epoch [10/10], Loss: 0.3923


In [5]:
train_loader1 = deal_dataloader(train_loader, model, device)

from models import modelset

# 定义损失函数和优化器
model2 = modelset.MLP(hidden_dim, num_classes).to(device)
criterion2 = nn.CrossEntropyLoss()  # 使用交叉熵损失
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)  # 使用随机梯度下降优化器

model2.train()
# 训练模型
epochs = 60
for epoch in range(epochs):
    for images, labels in train_loader1:
        # 将图像展平为一维向量，并将标签进行 one-hot 编码
        images = images.to(device)
        labels_one_hot = F.one_hot(labels, num_classes=num_classes).float().to(device)  # 将标签转换为 one-hot 编码

        # 前向传播
        outputs = model2(images)

        # 计算损失
        loss = criterion2(outputs, labels_one_hot)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/60], Loss: 0.7589
Epoch [2/60], Loss: 0.6577
Epoch [3/60], Loss: 0.7947
Epoch [4/60], Loss: 0.2216
Epoch [5/60], Loss: 0.0538
Epoch [6/60], Loss: 0.0208
Epoch [7/60], Loss: 0.2233
Epoch [8/60], Loss: 0.1038
Epoch [9/60], Loss: 0.0340
Epoch [10/60], Loss: 0.0357
Epoch [11/60], Loss: 0.0087
Epoch [12/60], Loss: 0.0095
Epoch [13/60], Loss: 0.0149
Epoch [14/60], Loss: 0.0140
Epoch [15/60], Loss: 0.0155
Epoch [16/60], Loss: 0.0090
Epoch [17/60], Loss: 0.0060
Epoch [18/60], Loss: 0.0112
Epoch [19/60], Loss: 0.0070
Epoch [20/60], Loss: 0.0034
Epoch [21/60], Loss: 0.0037
Epoch [22/60], Loss: 0.0057
Epoch [23/60], Loss: 0.0024
Epoch [24/60], Loss: 0.0033
Epoch [25/60], Loss: 0.0034
Epoch [26/60], Loss: 0.0022
Epoch [27/60], Loss: 0.0026
Epoch [28/60], Loss: 0.0025
Epoch [29/60], Loss: 0.0041
Epoch [30/60], Loss: 0.0037
Epoch [31/60], Loss: 0.0046
Epoch [32/60], Loss: 0.0020
Epoch [33/60], Loss: 0.0047
Epoch [34/60], Loss: 0.0022
Epoch [35/60], Loss: 0.0027
Epoch [36/60], Loss: 0.0008
E

In [6]:
import torch

# 设置模型为评估模式
model.eval()
model2.eval()

# 准确率计数
correct = 0
total = 0

# 禁用梯度计算，加速测试过程
with torch.no_grad():
    for images, labels in test_loader:
        # 将数据加载到 GPU
        images = images.view(-1, input_size).to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = model(images)
        outputs = model2(outputs)
        
        # 获取预测结果
        _, predicted = torch.max(outputs, 1)
        
        # 更新计数
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 计算准确率
accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy:.2f}%')

Accuracy on the test dataset: 89.02%


# 测试全连接的准确性

In [7]:
# 定义全连接层神经网络模型
class MLP2(nn.Module):
    def __init__(self, input_size, hidden_dim, num_classes):
        super(MLP2, self).__init__()
        self.input_size = input_size
        self.linear1 = nn.Linear(input_size, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.view(-1, self.input_size)  # 展平图像
        out = self.linear1(x)
        out = F.softmax(out)
        out = self.linear2(out)
        return out

In [8]:
# 定义损失函数和优化器
model3 = MLP2(input_size, 1000,num_classes).to(device)
criterion2 = nn.CrossEntropyLoss()  # 使用交叉熵损失
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)  # 使用随机梯度下降优化器

# 训练模型
epochs = 60
for epoch in range(epochs):
    for images, labels in train_loader:
        # 将图像展平为一维向量，并将标签进行 one-hot 编码
        images = images.to(device)
        labels_one_hot = F.one_hot(labels, num_classes=num_classes).float().to(device)  # 将标签转换为 one-hot 编码

        # 前向传播
        outputs = model3(images)

        # 计算损失
        loss = criterion2(outputs, labels_one_hot)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/60], Loss: 2.1345


/tmp/ipykernel_15580/3570110445.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


Epoch [2/60], Loss: 1.9214
Epoch [3/60], Loss: 1.6496


Epoch [4/60], Loss: 1.3822
Epoch [5/60], Loss: 1.3116
Epoch [6/60], Loss: 0.9886
Epoch [7/60], Loss: 0.9105
Epoch [8/60], Loss: 0.8328
Epoch [9/60], Loss: 0.6157
Epoch [10/60], Loss: 0.5488
Epoch [11/60], Loss: 0.5768
Epoch [12/60], Loss: 0.4642
Epoch [13/60], Loss: 0.5294
Epoch [14/60], Loss: 0.3732
Epoch [15/60], Loss: 0.2952
Epoch [16/60], Loss: 0.3365
Epoch [17/60], Loss: 0.3097
Epoch [18/60], Loss: 0.4922
Epoch [19/60], Loss: 0.2292
Epoch [20/60], Loss: 0.3184
Epoch [21/60], Loss: 0.2520
Epoch [22/60], Loss: 0.3107
Epoch [23/60], Loss: 0.2393
Epoch [24/60], Loss: 0.1354
Epoch [25/60], Loss: 0.2835
Epoch [26/60], Loss: 0.3091
Epoch [27/60], Loss: 0.3736
Epoch [28/60], Loss: 0.2371
Epoch [29/60], Loss: 0.1657
Epoch [30/60], Loss: 0.1071
Epoch [31/60], Loss: 0.0932
Epoch [32/60], Loss: 0.1548
Epoch [33/60], Loss: 0.1534
Epoch [34/60], Loss: 0.1418
Epoch [35/60], Loss: 0.0812
Epoch [36/60], Loss: 0.0708
Epoch [37/60], Loss: 0.3770
Epoch [38/60], Loss: 0.2029
Epoch [39/60], Loss: 0.067

In [9]:
import torch

# 设置模型为评估模式
model3.eval()

# 准确率计数
correct = 0
total = 0

# 禁用梯度计算，加速测试过程
with torch.no_grad():
    for images, labels in test_loader:
        # 将数据加载到 GPU
        images = images.view(-1, input_size).to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = model3(images)
        
        # 获取预测结果
        _, predicted = torch.max(outputs, 1)
        
        # 更新计数
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 计算准确率
accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy:.2f}%')

/tmp/ipykernel_15580/3570110445.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


Accuracy on the test dataset: 83.43%


上面逐个测试数据集在单层上面的准确性，并对比基于FBM和MLP两种情况下的表现）：
| Dataset       | Model | Accuracy  |
|---------------|-------|-----------|
| CIFAR-10      | FBM   | 29.15%    |
|               | MLP   | 26.47%    |
| KMNIST        | FBM   | 67.12%    |
|               | MLP   | 65.29%    |
| FashionMNIST  | FBM   | 79.89%    |
|               | MLP   | 75.10%    |
| MNIST         | FBM   | 88.83%    |
|               | MLP   | 85.07%    |

* 参数设置均为两层全连接网络，其中隐藏层为1000.
* 训练参数相同，epoch=60.训练集和测试集相同

并且发现，测试水平和数据集的选取有很大关系，选用不同的训练集，波动很大